# Learning and Decision Making

## Laboratory 3: Markov decision problems

In the end of the lab, you should submit all code/answers written in the tasks marked as "Activity n. XXX", together with the corresponding outputs and any replies to specific questions posed to the e-mail <adi.tecnico@gmail.com>. Make sure that the subject is of the form [&lt;group n.&gt;] LAB &lt;lab n.&gt;.

### 1. Modeling

Consider once again the predator-prey domain described in the Homework and which you described as a Markov decision process.

<img src="toroidal-world.png" width="400px">

Recall that:

* toroidal world "wraps around", i.e., an individual exiting through any of the four sides of the grid reenters on the opposite side (see figure above).

* At each time step, the hare selects uniformly at random one of the four directions (up, down, left, and right) and moves to the adjacent cell in that direction with a probability 0.4. With a probability 0.6 it remains in the same cell. 

* The wolf, on the other hand, can select at each time step one of five actions---up (_U_), down (_D_), left (_L_) and right (_R_) or stay (_S_). If it selects action _S_, it remains in the same cell with probability 1.0. Otherwise, the other 4 actions succeed in moving the wolf to the adjacent cell in the corresponding direction with a probability 0.8 and fail with a probability 0.2. 

* The goal of the wolf is to catch the hare.

---

#### Activity 1.        

Implement your Markov decision process in Python. In particular,

* Create a list with all the states;
* Create a list with all the actions;
* For each action, define a `numpy` array with the corresponding transition probabilities;
* Define a `numpy`array with the costs

The order for the states and actions used in the transition probability and cost matrices should match that in the lists of states and actions. 

**Note 1**: Don't forget to import `numpy`.

**Note 2**: You can define the transition probability matrices for each of the two individuals and then build the combined transition probability matrices using the `numpy.kron` function.

---

In [60]:
import numpy as np

import numpy as np



# Inicializa a lista X com [(1,1), (1,2), ...]
def setup_possible_states():
    possible_states = []
    for row in range(4):
        for column in range(4):
            current_state = (row+1, column+1)
            possible_states.append(current_state)
    return possible_states

# Diz onde vamos calhar se formos para cima ou para baixo (igual nos dois casos)
def calculate_up_down_result(position):
    return {
        1: 3,
        3: 1,
        2: 4,
        4: 2
    }[position]

# Diz onde vamos calhar se formos para a esquerda ou para a direita (igual nos dois casos)
def calculate_left_right_result(position):
    return {
        1: 2,
        2: 1,
        3: 4,
        4: 3
    }[position]

# Diz o indice da matrix 16x16 dado um estado (exemplo (1,1) = 1 | (1,2) = 2)
def state_to_matrix_index(x, y):
    index = 0
    for row in range(4):
        for column in range(4):
            if row+1 == x and column+1 == y:
                return index
            else:
                index += 1

# Devolve os estados possiveis e as probabilidades dos movimentos da lebre
# Apenas a posicao da lebre e considerada (lobo ignorado)
def possible_hare_positions_and_probabilities(hare_position):

    up_down_state = calculate_up_down_result(hare_position)
    left_right_state = calculate_left_right_result(hare_position)

    return {
        up_down_state : 0.4*0.5,
        left_right_state : 0.4*0.5,
        hare_position: (0.6*0.5*2)
    }

# Devolve os estados possiveis e as probabilidades quando o lobo fica parado
def possible_Stay_wolf_positions_and_probabilities(wolf_position, hare_position):

    dict = possible_hare_positions_and_probabilities(hare_position)
    states_and_probabilities = {}

    for hare_state in dict:
        current_state = (wolf_position, hare_state)
        states_and_probabilities[current_state] = dict[hare_state]

    return states_and_probabilities

# Devolve os estados possiveis e as probabilidades quando o lobo (tenta) ir para cima ou para baixo (iguais)
def possible_Up_Down_wolf_positions_and_probabilities(wolf_position, hare_position):

    states_and_probabilities = possible_Stay_wolf_positions_and_probabilities(wolf_position, hare_position)

    for state in states_and_probabilities:
        states_and_probabilities[state] = states_and_probabilities[state] * 0.2

    new_wolf_position_if_succeeded = calculate_up_down_result(wolf_position)
    dict = possible_hare_positions_and_probabilities(hare_position)
    for hare_state in dict:
        current_state = (new_wolf_position_if_succeeded, hare_state)
        states_and_probabilities[current_state] = dict[hare_state] * 0.8

    return states_and_probabilities

# Devolve os estados possiveis e as probabilidades quando o lobo (tenta) ir para a esquerda ou para a direita (iguais)
# Apenas muda uma linha em relacao a funcao acima
# Possivel refatorizar codigo duplicado!!!
def possible_Left_Right_wolf_positions_and_probabilities(wolf_position, hare_position):

    states_and_probabilities = possible_Stay_wolf_positions_and_probabilities(wolf_position, hare_position)

    for state in states_and_probabilities:
        states_and_probabilities[state] = states_and_probabilities[state] * 0.2

    new_wolf_position_if_succeeded = calculate_left_right_result(wolf_position)
    dict = possible_hare_positions_and_probabilities(hare_position)
    for hare_state in dict:
        current_state = (new_wolf_position_if_succeeded, hare_state)
        states_and_probabilities[current_state] = dict[hare_state] * 0.8

    return states_and_probabilities

# -------------------------------------------------------
# -------------------------------------------------------
# -------------------------------------------------------

#Estados Possiveis
X = setup_possible_states()

#Acoes Possiveis
A = ["U", "D", "L", "R", "S"]

#Matrizes de probabilidade de transicao para cada acao
P_Up = np.zeros((16, 16))
P_Down = np.zeros((16, 16))

P_Left = np.zeros((16, 16))
P_Right = np.zeros((16, 16))

P_Stay = np.zeros((16, 16))

# -----------------------------------------------------------------
# Preenchimento das matrizes de probabilidade de transicao --------
# -----------------------------------------------------------------

def fill_P_Up_Down(wolf_position, hare_position):
    states_and_probabilities = possible_Up_Down_wolf_positions_and_probabilities(wolf_position, hare_position)
    current_row = state_to_matrix_index(wolf_position, hare_position)
    for state in states_and_probabilities:
        current_column = state_to_matrix_index(state[0], state[1])
        P_Up[current_row][current_column] = states_and_probabilities[state]
        P_Down[current_row][current_column] = states_and_probabilities[state]

def fill_P_Left_Right(wolf_position, hare_position):
    states_and_probabilities = possible_Left_Right_wolf_positions_and_probabilities(wolf_position, hare_position)
    current_row = state_to_matrix_index(wolf_position, hare_position)
    for state in states_and_probabilities:
        current_column = state_to_matrix_index(state[0], state[1])
        P_Left[current_row][current_column] = states_and_probabilities[state]
        P_Right[current_row][current_column] = states_and_probabilities[state]

def fill_P_Stay(wolf_position, hare_position):
    states_and_probabilities = possible_Stay_wolf_positions_and_probabilities(wolf_position, hare_position)
    current_row = state_to_matrix_index(wolf_position, hare_position)
    for state in states_and_probabilities:
        current_column = state_to_matrix_index(state[0], state[1])
        P_Stay[current_row][current_column] = states_and_probabilities[state]

def build_transition_probability_matrices():
    for row in range(4):
        for column in range(4):

            wolf_position = row+1
            hare_position = column+1

            fill_P_Up_Down(wolf_position, hare_position)
            fill_P_Left_Right(wolf_position, hare_position)
            fill_P_Stay(wolf_position, hare_position)


# Preencher Matrizes de Transicao para as acoes
build_transition_probability_matrices()
#Acoes Possiveis
#A = ["U", "D", "L", "R", "S"]


###################################################################
###############  Matrix de Custo   ################################
###################################################################

same_position_state_indexes = [0, 5, 10, 15]

above_bellow_position_state_indexes = [2, 7, 8, 13]

left_right_position_state_indexes = [1, 4, 11, 14]

diagonal_position_state_indexes = [3, 6, 9, 12]

stay = 4

up_or_down = [0, 1]

left_or_right = [2, 3]

def build_cost_matrix():
    M = np.zeros((16, 5))
    for possible_action in range(5):
        for possible_state in range(16):
            if possible_state in same_position_state_indexes and possible_action == stay:
                M[possible_state, possible_action] = 0
                
            elif possible_state in above_bellow_position_state_indexes and possible_action in up_or_down:
                M[possible_state, possible_action] = 0
            
            elif possible_state in left_right_position_state_indexes and possible_action in left_or_right:
                M[possible_state, possible_action] = 0
                
            elif possible_state in diagonal_position_state_indexes and possible_action != stay:
                M[possible_state, possible_action] = 0
                
            else:
                M[possible_state, possible_action] = 1
                
    return costMatrix

costMatrix = build_cost_matrix()

print(costMatrix)







[[ 1.  1.  1.  1.  0.]
 [ 1.  1.  0.  0.  1.]
 [ 0.  0.  1.  1.  1.]
 [ 0.  0.  0.  0.  1.]
 [ 1.  1.  0.  0.  1.]
 [ 1.  1.  1.  1.  0.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  1.  1.  1.]
 [ 0.  0.  1.  1.  1.]
 [ 0.  0.  0.  0.  1.]
 [ 1.  1.  1.  1.  0.]
 [ 1.  1.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  1.  1.  1.]
 [ 1.  1.  0.  0.  1.]
 [ 1.  1.  1.  1.  0.]]


### 2. Prediction

You are now going to evaluate a given policy, computing the corresponding cost-to-go.

---

#### Activity 2.

Describe the policy that, in each state $(w, h)$, always moves the wolf to the cell closest to the hare. If multiple such cells exist, the wolf should select randomly between the two.

For example, suppose that the wolf is in cell 1 and the hare in cell 4 (figure above, left). The wolf should then select randomly between the actions _U_, _D_ (which move the wolf to cell 3), _L_ and _R_ (which move the wolf to cell 2). Conversely, if the wolf is in cell 1 and the hare in cell 3 (figure above, right), the wolf should select randomly between the two actions _U_ and _D_ (which move the wolf to cell 3).

**Note:** The policy should be described as a vector with as many rows as there are states and as many columns as there are actions, where the entry _xa_ has the probability of selecting action _a_ in state _x_.

---

In [52]:
import numpy as np

same_position_state_indexes = [0, 5, 10, 15]

above_bellow_position_state_indexes = [2, 7, 8, 13]

left_right_position_state_indexes = [1, 4, 11, 14]

diagonal_position_state_indexes = [3, 6, 9, 12]

stay = 4

up_or_down = [0, 1]

left_or_right = [2, 3]

def build_cost_matrix():
    costMatrix = np.zeros((16, 5))
    for possible_action in range(5):
        for possible_state in range(16):
            if possible_state in same_position_state_indexes and possible_action == stay:
                costMatrix[possible_state, possible_action] = 1
                
            elif possible_state in above_bellow_position_state_indexes and possible_action in up_or_down:
                costMatrix[possible_state, possible_action] = 0.5
            
            elif possible_state in left_right_position_state_indexes and possible_action in left_or_right:
                costMatrix[possible_state, possible_action] = 0.5
                
            elif possible_state in diagonal_position_state_indexes and possible_action != stay:
                costMatrix[possible_state, possible_action] = 0.25
                
            else:
                costMatrix[possible_state, possible_action] = 0
                
    print(costMatrix)

build_cost_matrix()








[[ 0.    0.    0.    0.    1.  ]
 [ 0.    0.    0.5   0.5   0.  ]
 [ 0.5   0.5   0.    0.    0.  ]
 [ 0.25  0.25  0.25  0.25  0.  ]
 [ 0.    0.    0.5   0.5   0.  ]
 [ 0.    0.    0.    0.    1.  ]
 [ 0.25  0.25  0.25  0.25  0.  ]
 [ 0.5   0.5   0.    0.    0.  ]
 [ 0.5   0.5   0.    0.    0.  ]
 [ 0.25  0.25  0.25  0.25  0.  ]
 [ 0.    0.    0.    0.    1.  ]
 [ 0.    0.    0.5   0.5   0.  ]
 [ 0.25  0.25  0.25  0.25  0.  ]
 [ 0.5   0.5   0.    0.    0.  ]
 [ 0.    0.    0.5   0.5   0.  ]
 [ 0.    0.    0.    0.    1.  ]]


---

#### Activity 3.

Compute the cost-to-go function $J^\pi$ associated with the policy from Activity 2. Use $\gamma=0.99$.

---

In [69]:
import numpy as np
from numpy.linalg import inv



ident = np.matrix([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])


# matrix de custos em principio sera aquela que ja criamos ??



# Falta a matrix de probabilidades que sera uma combinaçao das matrizes de todas as accoes ??

gamma = 0.99



def calc_cost_to_go(prob_matrix):
    
    #devolve vector custo da acçao up ou down
    if np.all(prob_matrix == P_Up) or np.all(prob_matrix == P_Down):
        cost_vector = costMatrix[:,0]
    
    #devolve vector custo da acçao left ou right
    elif np.all(prob_matrix == P_Left) or np.all(prob_matrix == P_Right):
        cost_vector = costMatrix[:,2]
        
    #devolve vector custo da acçao stay
    else:
        cost_vector = costMatrix[:,4]
    
    aux = ident - (np.dot(gamma, probMatrix))

    invMatrix = inv(aux)
    
    return np.dot(invMatrix, cost_vector)



J_Up = calc_cost_to_go(P_Up)
J_Down = calc_cost_to_go(P_Down)
J_Left = calc_cost_to_go(P_Left)
J_Right = calc_cost_to_go(P_Right)
J_Stay = calc_cost_to_go(P_Stay)

print(J_Stay)


[[ 73.97647134  75.65490583  74.79200157  75.57662126  75.65490583
   73.97647134  75.57662126  74.79200157  74.79200157  75.57662126
   73.97647134  75.65490583  75.57662126  74.79200157  75.65490583
   73.97647134]]


### 3. Control

In this section you are going to compare value and policy iteration, both in terms of time and number of iterations.

---

#### Activity 4

Show that the policy in Activity 3 is optimal: use value iteration to compute $J^*$ and show that $J^*=J^\pi$. Track the time and the number of iterations taken to compute $J^*$.

**Note 1:** Stop the algorithm when the error between iterations is smaller than $10^{-8}$.

**Note 2:** You may find useful the function ``time()`` from the module ``time``.

---

In [44]:
m1 = np.array([[1, 2], [3, 4]])
m2 = np.array([[1, 2], [3, 4]])

print(m1*m2)
print(np.dot(m1,m2))

[[ 1  4]
 [ 9 16]]
[[ 7 10]
 [15 22]]


---

#### Activity 5

Compute once again the optimal policy now using policy iteration. Track the time and number of iterations taken and compare to those of Activity 4.

**Note:** If you find that numerical errors affect your computations (especially when comparing two values/arrays) you may use the `numpy` function `isclose` with adequately set absolute and relative tolerance parameters (e.g., $10^{-8}$).

---

In [5]:
# Insert your code here

### 4. Simulation

Finally, in this section you will check whether the theoretical computations of the cost-to-go actually correspond to the cost incurred by an agent following a policy.

---

#### Activity 6

Starting in each of the two states $x$ in the initial figure, 

* Generate **100** trajectories of 10,000 steps each, following the optimal policy for the MDP. 
* For each trajectory, compute the accumulated (discounted) cost. 
* Compute the average cost over the 100 trajectories.
* Compare the resulting value with that computed in Activity 4 for the two states. 

** Note:** The simulation may take a bit of time, don't despair ☺️.

---

In [6]:
# Insert your code here